Police crime data for San Francisco can be obtained from: https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-2018-to-Present/wg3w-h783

The strategy for data processing and features:

1. Select: Incident Date, Incident Category

2. Handling Missing Values:
For Incident Category rows with missing values will be dropped or filled with "Unkown"

3. Data Transformation:
Incident Date will be converted to a datetime object, and additional time-based features such as the day of the week, month, and year will be extracted.

4. Feature Engineering:
Aggregate incidents daily, count the number of incidents of each type.
Calculate lag features to incorporate historical data, such as the number of incidents in the previous days.
Define a hotspot based on whether the next day’s incidents exceed one standard deviation above the mean of the previous 60 days
5. Preparing the Target Variable:
The target variable will be whether the next day is a hotspot, defined based on the criteria

In [1]:
import pandas as pd
import numpy as np

In [2]:
# make sure to install this package before running:
#!pip install sodapy
#!pip install scikit-learn
#!pip install statsmodels

In [3]:
#test

# San Francisco Police Crime Data 
# Access the dataset via API

Currently works without a token. It queries only 2000 records. We need to connect with a token so we can query every night. This dataset is updated every night at 10pm.

In [4]:
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.sfgov.org", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.sfgov.org,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
SFC = client.get("wg3w-h783", limit=10000)

# Convert to pandas DataFrame
SFC_df = pd.DataFrame.from_records(SFC)

# Santa Clara County Sheriff's Office incident dataset

# Access the dataset via API

In [5]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.sccgov.org", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.sccgov.org,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
SCC = client.get("n9u6-aijz", limit=2000)

# Convert to pandas DataFrame
SCC_df = pd.DataFrame.from_records(SCC)

# Load the full CSV dataset. Last download 11/30/2023

In [6]:
# Load the SF data
sf_file_path = 'data/SF_Crime_Reports.csv'
sf_data = pd.read_csv(sf_file_path)

# EDA

In [7]:
sf_data.columns

Index(['Incident Datetime', 'Incident Date', 'Incident Time', 'Incident Year',
       'Incident Day of Week', 'Report Datetime', 'Row ID', 'Incident ID',
       'Incident Number', 'CAD Number', 'Report Type Code',
       'Report Type Description', 'Filed Online', 'Incident Code',
       'Incident Category', 'Incident Subcategory', 'Incident Description',
       'Resolution', 'Intersection', 'CNN', 'Police District',
       'Analysis Neighborhood', 'Supervisor District',
       'Supervisor District 2012', 'Latitude', 'Longitude', 'Point',
       'Neighborhoods', 'ESNCAG - Boundary File',
       'Central Market/Tenderloin Boundary Polygon - Updated',
       'Civic Center Harm Reduction Project Boundary',
       'HSOC Zones as of 2018-06-05', 'Invest In Neighborhoods (IIN) Areas',
       'Current Supervisor Districts', 'Current Police Districts'],
      dtype='object')

### CLEAN SF DATA

In [8]:
#test

In [9]:
sf_data = sf_data[['Incident Date', 'Incident Time', 'Incident Year',
       'Incident Day of Week', 'Incident Code',
       'Incident Category', 'Incident Subcategory', 'Incident Description',
       'Intersection', 'Police District',
       'Analysis Neighborhood', 'Latitude', 'Longitude', 'Point',
       'Neighborhoods']] 

sf_data = sf_data.dropna()

sf_data.head()

,Incident Date,Incident Time,Incident Year,Incident Day of Week,Incident Code,Incident Category,Incident Subcategory,Incident Description,Intersection,Police District,Analysis Neighborhood,Latitude,Longitude,Point,Neighborhoods
11,2022/06/27,12:00,2022,Monday,71000,Lost Property,Lost Property,Lost Property,GEARY ST \ POWELL ST,Central,Financial District/South Beach,37.787359,-122.408227,POINT (-122.40822672700406 37.78735926098589),19.0
13,2023/03/16,17:30,2023,Thursday,4134,Assault,Simple Assault,Battery,18TH ST \ DE HARO ST,Bayview,Potrero Hill,37.762290,-122.401324,POINT (-122.40132418490647 37.76228996810526),54.0
33,2023/03/21,15:50,2023,Tuesday,51040,Non-Criminal,Non-Criminal,Aided Case,POST ST \ LARKIN ST,Northern,Tenderloin,37.787038,-122.418271,POINT (-122.41827098126804 37.787037946181535),50.0
61,2021/08/22,09:40,2021,Sunday,62071,Warrant,Other,Probation Search,LAGUNA ST \ PACIFIC AVE,Northern,Pacific Heights,37.793977,-122.429804,POINT (-122.42980398313114 37.79397724418211),102.0
87,2022/07/02,22:53,2022,Saturday,4134,Assault,Simple Assault,Battery,GILMAN AVE \ HAWES ST,Bayview,Bayview Hunters Point,37.719298,-122.390020,POINT (-122.39001968579463 37.719298464887245),88.0


In [10]:
sf_data.columns = sf_data.columns.str.replace(' ', '_')
sf_data.head()

,Incident_Date,Incident_Time,Incident_Year,Incident_Day_of_Week,Incident_Code,Incident_Category,Incident_Subcategory,Incident_Description,Intersection,Police_District,Analysis_Neighborhood,Latitude,Longitude,Point,Neighborhoods
11,2022/06/27,12:00,2022,Monday,71000,Lost Property,Lost Property,Lost Property,GEARY ST \ POWELL ST,Central,Financial District/South Beach,37.787359,-122.408227,POINT (-122.40822672700406 37.78735926098589),19.0
13,2023/03/16,17:30,2023,Thursday,4134,Assault,Simple Assault,Battery,18TH ST \ DE HARO ST,Bayview,Potrero Hill,37.762290,-122.401324,POINT (-122.40132418490647 37.76228996810526),54.0
33,2023/03/21,15:50,2023,Tuesday,51040,Non-Criminal,Non-Criminal,Aided Case,POST ST \ LARKIN ST,Northern,Tenderloin,37.787038,-122.418271,POINT (-122.41827098126804 37.787037946181535),50.0
61,2021/08/22,09:40,2021,Sunday,62071,Warrant,Other,Probation Search,LAGUNA ST \ PACIFIC AVE,Northern,Pacific Heights,37.793977,-122.429804,POINT (-122.42980398313114 37.79397724418211),102.0
87,2022/07/02,22:53,2022,Saturday,4134,Assault,Simple Assault,Battery,GILMAN AVE \ HAWES ST,Bayview,Bayview Hunters Point,37.719298,-122.390020,POINT (-122.39001968579463 37.719298464887245),88.0


In [11]:
count= sf_data['Incident_Code'].nunique()
count

815

In [12]:
sf_data.describe()

,Incident_Year,Incident_Code,Latitude,Longitude,Neighborhoods
count,739563.000000,739563.000000,739563.000000,739563.000000,739563.000000
mean,2020.382116,24836.094292,37.769069,-122.423924,53.089376
std,1.726014,25676.018765,0.024547,0.026472,32.636531
min,2018.000000,1000.000000,37.708256,-122.511295,1.000000
25%,2019.000000,6244.000000,37.754146,-122.434350,23.000000
50%,2020.000000,7045.000000,37.775695,-122.417707,50.000000
75%,2022.000000,51040.000000,37.785759,-122.407255,86.000000
max,2023.000000,75030.000000,37.829991,-122.363743,117.000000


In [13]:
from sklearn.model_selection import train_test_split

sf_data_train, sf_data_test = train_test_split(sf_data, test_size=0.3, random_state=88)
sf_data_train.shape, sf_data_test.shape

sf_data_train.head()

,Incident_Date,Incident_Time,Incident_Year,Incident_Day_of_Week,Incident_Code,Incident_Category,Incident_Subcategory,Incident_Description,Intersection,Police_District,Analysis_Neighborhood,Latitude,Longitude,Point,Neighborhoods
188312,2022/01/23,15:00,2022,Sunday,6373,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, $200-$950",FILBERT ST \ STOCKTON ST,Central,North Beach,37.801371,-122.409369,POINT (-122.40936911818164 37.8013706547635),106.0
406737,2018/07/13,17:15,2018,Friday,72000,Non-Criminal,Non-Criminal,Found Property,FRANCISCO ST \ GOUGH ST,Northern,Marina,37.802791,-122.428210,POINT (-122.42821030774003 37.80279069007599),17.0
167872,2022/05/09,07:19,2022,Monday,27195,Other Miscellaneous,Trespass,Trespassing,ELLIS ST \ CYRIL MAGNIN ST,Tenderloin,Tenderloin,37.785388,-122.408667,POINT (-122.40866708530758 37.7853882479001),19.0
580466,2019/07/31,09:30,2019,Wednesday,4134,Assault,Simple Assault,Battery,VAN DYKE AVE \ JENNINGS ST,Bayview,Bayview Hunters Point,37.727044,-122.388673,POINT (-122.38867281318191 37.72704414579369),86.0
786654,2021/05/27,03:00,2021,Thursday,5082,Burglary,Burglary - Hot Prowl,"Burglary, Hot Prowl, Att. Forcible Entry",PIERCE ST \ GREENWICH ST,Northern,Marina,37.798533,-122.439156,POINT (-122.43915622018719 37.79853332699335),14.0


Base Line Model - SF Data

In [14]:
#Dummy Variables - Starting off with 2 incident codes on the train and test specifically
#Train: 65010: Traffic Violation Arrest; 4134: Assault
#Test: 6244: Larceny Theft: Locked Vehicle; 64070: Suspicious Occ


sf_data_train2 = sf_data_train.copy() 
sf_data_train2['IC65010'] = (sf_data_train2['Incident_Code'] == 65010).astype('int64')
sf_data_train2['IC4134'] = (sf_data_train2['Incident_Code'] == 4134).astype('int64')

sf_data_test2 = sf_data_test.copy() 
sf_data_test2['IC6244'] = (sf_data_test2['Incident_Code'] == 6244).astype('int64')
sf_data_test2['IC64070'] = (sf_data_test2['Incident_Code'] == 64070).astype('int64')

sf_data_test2.head()

,Incident_Date,Incident_Time,Incident_Year,Incident_Day_of_Week,Incident_Code,Incident_Category,Incident_Subcategory,Incident_Description,Intersection,Police_District,Analysis_Neighborhood,Latitude,Longitude,Point,Neighborhoods,IC6244,IC64070
290687,2022/11/30,12:54,2022,Wednesday,7055,Motor Vehicle Theft,Motor Vehicle Theft,"Vehicle, Recovered, Stolen outside SF",MISSOURI ST \ WATCHMAN WAY,Bayview,Potrero Hill,37.755621,-122.395763,POINT (-122.3957626269508 37.75562144550937),54.0,0,0
22461,2023/04/23,23:52,2023,Sunday,5043,Burglary,Burglary - Residential,"Burglary, Residence, Unlawful Entry",PACIFIC AVE \ BURGOYNE ST,Central,Russian Hill,37.795626,-122.417005,POINT (-122.41700510898492 37.79562571361739),16.0,0,0
551588,2019/06/05,17:50,2019,Wednesday,6364,Larceny Theft,Larceny Theft - Shoplifting,"Theft, Shoplifting, >$950",STOCKTON ST \ POST ST,Central,Financial District/South Beach,37.788499,-122.406771,POINT (-122.4067711451779 37.788499477941336),19.0,0,0
323057,2021/04/29,16:00,2021,Thursday,7043,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Motorcycle",WOODSIDE AVE \ HERNANDEZ AVE,Taraval,West of Twin Peaks,37.746328,-122.455467,POINT (-122.45546673383409 37.746327979628276),49.0,0,0
146603,2021/08/02,23:50,2021,Monday,7021,Motor Vehicle Theft,Motor Vehicle Theft,"Vehicle, Stolen, Auto",CUSTER AVE \ QUINT ST,Bayview,Bayview Hunters Point,37.745298,-122.389193,POINT (-122.38919329994268 37.74529840818052),56.0,0,0


In [15]:
ic_65010 = np.sum(sf_data_train2['IC65010'] == 1)  # total number of 65010 cases
ic_4134 = np.sum(sf_data_train2['IC4134'] == 1)   # total number of 4134 cases

print(pd.Series({'0': ic_65010, '1': ic_4134}))

0     5088
1    16269
dtype: int64


In [16]:
sf_data

,Incident_Date,Incident_Time,Incident_Year,Incident_Day_of_Week,Incident_Code,Incident_Category,Incident_Subcategory,Incident_Description,Intersection,Police_District,Analysis_Neighborhood,Latitude,Longitude,Point,Neighborhoods
11,2022/06/27,12:00,2022,Monday,71000,Lost Property,Lost Property,Lost Property,GEARY ST \ POWELL ST,Central,Financial District/South Beach,37.787359,-122.408227,POINT (-122.40822672700406 37.78735926098589),19.0
13,2023/03/16,17:30,2023,Thursday,4134,Assault,Simple Assault,Battery,18TH ST \ DE HARO ST,Bayview,Potrero Hill,37.762290,-122.401324,POINT (-122.40132418490647 37.76228996810526),54.0
33,2023/03/21,15:50,2023,Tuesday,51040,Non-Criminal,Non-Criminal,Aided Case,POST ST \ LARKIN ST,Northern,Tenderloin,37.787038,-122.418271,POINT (-122.41827098126804 37.787037946181535),50.0
61,2021/08/22,09:40,2021,Sunday,62071,Warrant,Other,Probation Search,LAGUNA ST \ PACIFIC AVE,Northern,Pacific Heights,37.793977,-122.429804,POINT (-122.42980398313114 37.79397724418211),102.0
87,2022/07/02,22:53,2022,Saturday,4134,Assault,Simple Assault,Battery,GILMAN AVE \ HAWES ST,Bayview,Bayview Hunters Point,37.719298,-122.390020,POINT (-122.39001968579463 37.719298464887245),88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799662,2023/03/22,13:30,2023,Wednesday,3401,Robbery,Robbery - Commercial,"Shoplifting, Force against Agent",24TH ST \ POTRERO AVE,Mission,Mission,37.753004,-122.406336,POINT (-122.40633623830558 37.75300402962228),54.0
799663,2023/03/22,00:13,2023,Wednesday,6371,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, <$50",SACRAMENTO ST \ DIVISADERO ST,Northern,Pacific Heights,37.788995,-122.440624,POINT (-122.44062361999508 37.78899525864335),102.0
799664,2023/03/21,21:19,2023,Tuesday,72000,Non-Criminal,Non-Criminal,Found Property,MISSION ST \ ACTON ST \ SICKLES AVE,Ingleside,Outer Mission,37.708806,-122.452665,POINT (-122.45266472219633 37.70880633652071),66.0
799665,2023/03/22,15:28,2023,Wednesday,6303,Larceny Theft,Larceny Theft - From Building,"Theft, From Building, $200-$950",BUSH ST \ GRANT AVE,Central,Financial District/South Beach,37.790565,-122.405574,POINT (-122.40557374633903 37.790565236529574),19.0


In [17]:
print(sf_data_test2.columns)

Index(['Incident_Date', 'Incident_Time', 'Incident_Year',
       'Incident_Day_of_Week', 'Incident_Code', 'Incident_Category',
       'Incident_Subcategory', 'Incident_Description', 'Intersection',
       'Police_District', 'Analysis_Neighborhood', 'Latitude', 'Longitude',
       'Point', 'Neighborhoods', 'IC6244', 'IC64070'],
      dtype='object')


In [18]:
import re

# Create a copy
df_copy = sf_data_test2.copy()
# Regular expression pattern for date
Pattern_data= r'(\d{4}/\d{2}/(\d{2}))'

# Extracting regex and format 'Incident_Time'
df_copy['Day'] = df_copy['Incident_Date'].str.split('/').str[-1].astype(int)
df_copy['Month'] = df_copy['Incident_Date'].str.split('/').str[1].astype(int)
df_copy['Incident_Time'] = pd.to_datetime(df_copy['Incident_Time'], format='%H:%M')

# Convert time to minutes past midnight
df_copy['Time_in_Minutes'] = df_copy['Incident_Time'].dt.hour * 60 + df_copy['Incident_Time'].dt.minute

# One-hot encode 'Incident_Day_of_Week'
one_hot_encoded_data = pd.get_dummies(df_copy['Incident_Day_of_Week'], prefix='Day')

# Convert one-hot encoded data to integer type
one_hot_encoded_data = one_hot_encoded_data.astype(int)

# Join the one-hot encoded data with the original df
df_copy = df_copy.join(one_hot_encoded_data)

# Drop the unnecessary columns from the DataFrame
df_copy = df_copy.drop(columns=['Point', 'Incident_Day_of_Week','Incident_Category', 'Incident_Date', 'Incident_Subcategory', 'Incident_Description', 'Intersection', 'Police_District', 'Analysis_Neighborhood'])

df = df_copy

df

,Incident_Time,Incident_Year,Incident_Code,Latitude,Longitude,Neighborhoods,IC6244,IC64070,Day,Month,Time_in_Minutes,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday
290687,1900-01-01 12:54:00,2022,7055,37.755621,-122.395763,54.0,0,0,30,11,774,0,0,0,0,0,0,1
22461,1900-01-01 23:52:00,2023,5043,37.795626,-122.417005,16.0,0,0,23,4,1432,0,0,0,1,0,0,0
551588,1900-01-01 17:50:00,2019,6364,37.788499,-122.406771,19.0,0,0,5,6,1070,0,0,0,0,0,0,1
323057,1900-01-01 16:00:00,2021,7043,37.746328,-122.455467,49.0,0,0,29,4,960,0,0,0,0,1,0,0
146603,1900-01-01 23:50:00,2021,7021,37.745298,-122.389193,56.0,0,0,2,8,1430,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179963,1900-01-01 19:00:00,2021,6244,37.785564,-122.429783,101.0,1,0,15,12,1140,0,0,0,0,0,0,1
613865,1900-01-01 17:35:00,2019,19057,37.779212,-122.410937,32.0,0,0,30,12,1055,0,1,0,0,0,0,0
332923,1900-01-01 18:00:00,2018,9029,37.790833,-122.410998,16.0,0,0,22,5,1080,0,0,0,0,0,1,0
507041,1900-01-01 18:45:00,2018,6374,37.784453,-122.408493,19.0,0,0,30,8,1125,0,0,0,0,1,0,0


In [25]:
import statsmodels.formula.api as smf

# IC6244 is dependent variable
formula = 'IC6244 ~ Incident_Year + Incident_Code + Latitude + Longitude + Neighborhoods + Day + Month + Time_in_Minutes + Day_Friday + Day_Monday + Day_Saturday + Day_Sunday + Day_Thursday + Day_Tuesday + Day_Wednesday'

# Fit the logistic regression model
logit_model = smf.logit(formula=formula, data=df).fit()

# Print the summary of the model
print(logit_model.summary())

Optimization terminated successfully.
         Current function value: 0.274673
         Iterations 14
                           Logit Regression Results                           
Dep. Variable:                 IC6244   No. Observations:               221869
Model:                          Logit   Df Residuals:                   221854
Method:                           MLE   Df Model:                           14
Date:                Sun, 03 Dec 2023   Pseudo R-squ.:                  0.2317
Time:                        19:27:31   Log-Likelihood:                -60941.
converged:                       True   LL-Null:                       -79323.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept       -2176.1027   3.26e+05     -0.007      0.995   -6.42e+05    6.38e+05
Incident_Year

In [26]:
import statsmodels.formula.api as smf

# IC6244 is dependent variable
formula = 'IC64070 ~ Incident_Year + Incident_Code + Latitude + Longitude + Neighborhoods + Day + Month + Time_in_Minutes + Day_Friday + Day_Monday + Day_Saturday + Day_Sunday + Day_Thursday + Day_Tuesday + Day_Wednesday'

# Fit the logistic regression model
logit_model = smf.logit(formula=formula, data=df).fit()

# Print the summary of the model
print(logit_model.summary())

Optimization terminated successfully.
         Current function value: 0.075632
         Iterations 18
                           Logit Regression Results                           
Dep. Variable:                IC64070   No. Observations:               221869
Model:                          Logit   Df Residuals:                   221854
Method:                           MLE   Df Model:                           14
Date:                Sun, 03 Dec 2023   Pseudo R-squ.:                  0.2235
Time:                        19:31:34   Log-Likelihood:                -16780.
converged:                       True   LL-Null:                       -21610.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        -197.8556   6.38e+05     -0.000      1.000   -1.25e+06    1.25e+06
Incident_Year

In [27]:
first_row = df.dtypes
print(first_row)

Incident_Time      datetime64[ns]
Incident_Year               int64
Incident_Code               int64
Latitude                  float64
Longitude                 float64
Neighborhoods             float64
IC6244                      int64
IC64070                     int64
Day                         int32
Month                       int32
Time_in_Minutes             int32
Day_Friday                  int32
Day_Monday                  int32
Day_Saturday                int32
Day_Sunday                  int32
Day_Thursday                int32
Day_Tuesday                 int32
Day_Wednesday               int32
dtype: object
